# KeT5를 활용하여 Translation 구현하기 (Large Version)

Author : 정상근 (hugmanskj@gmail.com)

> 알림>
> 교육용 목적으로 제작된 코드입니다.

이 노트북에서는 한국어 T5모델 중 하나인 KeT5 이용하여 아주 간단한 한국어-영어 번역 문제를 풀어보겠습니다.

활용 데이터는 "bongsoo/news_talk_en_ko" 데이터를 이용하여 진행하겠습니다.

아래의 소프트웨어와 프레임워크를 사용해서 진행하겠습니다.

(훈련 완료 시간 : 4시간)

In [1]:
# Performance Metric
!pip install -q sacrebleu


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
from IPython.display import Markdown, display

In [3]:
import os

# 첫 번째 GPU만 사용하도록 설정 (본 실습환경 때문)
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

## Dataset 준비

In [4]:
# TRUE -> 모든 데이터 사용
# FALSE -> 테스트를 위한 소규모 데이터만 사용
#USE_ALL_DATA = False
USE_ALL_DATA = True

In [5]:
## It takes time! (about 10min)
from datasets import load_dataset, Dataset, DatasetDict
import pandas as pd

# the following code is sometimes not working at AWS.
#dataset = load_dataset("bongsoo/news_talk_en_ko", column_names=["english", "korean"])

ori_dataset = load_dataset("bongsoo/news_talk_en_ko")
def correct_dataform(from_data):
    from_data.set_format(type="pandas")
    df = from_data["train"][:]
    example_0 = list(df.columns)
    example_0_df = pd.DataFrame({col: [value] for col, value in zip(('english', 'korean'), example_0)})

    df.columns = ('english', 'korean')
    en_ko_df = pd.concat([example_0_df, df],).reset_index(drop=True)
    en_ko_df.head()
    dataset = Dataset.from_pandas(en_ko_df)
    return DatasetDict({
                        "train": dataset
                    })
dataset = correct_dataform(ori_dataset)

### Dataset 살펴보기

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['english', 'korean'],
        num_rows: 1300000
    })
})

In [7]:
dataset['train']['english'][:10]

["Skinner's reward is mostly eye-watering.",
 'Even some problems can be predicted.',
 'Only God will exactly know why.',
 "Businesses should not overlook China's dispute.",
 'Slow-beating songs often float over time.',
 "I can't even consider uninsured treatments.",
 'Including Jesus, thirteen people eat together.',
 'Uncertified companies should use government-created platforms.',
 "At least someone won't be missed.",
 "Children's evaluations show that they work."]

In [8]:
dataset['train']['korean'][:10]

['스키너가 말한 보상은 대부분 눈으로 볼 수 있는 현물이다.',
 '심지어 어떤 문제가 발생할 건지도 어느 정도 예측이 가능하다.',
 '오직 하나님만이 그 이유를 제대로 알 수 있을 겁니다.',
 '중국의 논쟁을 보며 간과해선 안 될 게 기업들의 고충이다.',
 '박자가 느린 노래는 오랜 시간이 지나 뜨는 경우가 있다.',
 '보험 처리가 안 되는 비급여 시술은 엄두도 못 낸다.',
 '예수까지 합치면 모두 열세 명이 함께 식사를 하는 것이다.',
 '인증을 받지 못한 기업은 정부가 만든 플랫폼을 활용해야 한다.',
 '적어도 누군가 보고 싶은 일이 일어나진 않을 듯 합니다.',
 '아이들 평가를 해보면 효과가 있다는 것을 알 수 있다.']

### Train / Valid / Test 분할하기

In [9]:
from datasets import DatasetDict
# 데이터셋을 훈련, 검증, 테스트 세트로 나누기
train_testvalid = dataset["train"].train_test_split(test_size=0.2)
test_valid = train_testvalid['test'].train_test_split(test_size=0.5)

# 최종적으로 train, valid, test 세트 생성
train_dataset = train_testvalid['train']
valid_dataset = test_valid['train']
test_dataset = test_valid['test']

# DatasetDict로 조합
datasets = DatasetDict({
    "train": train_dataset,
    "validation": valid_dataset,
    "test": test_dataset
})

In [10]:
datasets

DatasetDict({
    train: Dataset({
        features: ['english', 'korean'],
        num_rows: 1040000
    })
    validation: Dataset({
        features: ['english', 'korean'],
        num_rows: 130000
    })
    test: Dataset({
        features: ['english', 'korean'],
        num_rows: 130000
    })
})

적절히 잘 나눠졌음을 확인할 수 있습니다.

### Data 전처리 하기

### Model 설정하기

In [11]:
MODEL_ID = "KETI-AIR/ke-t5-base"

In [12]:
import transformers
from transformers import AutoTokenizer    
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=False)

# fast version 인지 아닌지 확인
#assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [13]:
max_token_length = 64

In [14]:
#print(tokenizer.vocab)

보통은 Encoder - Decoder 모델에 있어서, Target sequence output을 만들 때 아래처럼, tokenizer를 target langauge에 맞게 바꿔줄 필요가 있습니다. 
> tokenizer.src_lang = 'enr-XX'

> tokenizer.tgt_lang = 'kr-KR'

하지만 우리가 사용하는 모델은 같은 Vocab file을 사용하기 때문에 굳이 해당 루틴을 적용할 필요는 없습니다.

In [15]:
e = dataset["train"]['english'][0]
k = dataset["train"]['korean'][0]

e_output = tokenizer(e)
with tokenizer.as_target_tokenizer():
    k_output = tokenizer(k)

print( e_output['input_ids']  )
print( k_output['input_ids'] )
print("-"*50)
print( tokenizer.decode( e_output['input_ids'] ))
print( tokenizer.decode( k_output['input_ids'] ))

[388, 6809, 2952, 17, 8, 32204, 43, 8023, 6687, 28, 9495, 91, 3, 1]
[13946, 1800, 22, 4937, 4810, 19, 1657, 11023, 487, 35, 53, 26911, 131, 3, 1]
--------------------------------------------------
Skinner's reward is mostly eye-watering.</s>
스키너가 말한 보상은 대부분 눈으로 볼 수 있는 현물이다.</s>


/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3951: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


> 위 결과에서 보듯이, 이번에 사용하는 ""KETI-AIR/ke-t5-base" 모델은 Encoder, Decoder 모두 같은 vocabulary 를 사용하고 있는것을 알 수 있습니다.

### Prefix 넣어주기
T5 original paper에서는 모든 Task 에 task prefix를 넣도록 하는 방법을 제안합니다. 즉, 영어에서 한국어로 번역한다면 "Translate English to Korean : "이라는 prefix term 을 입력의 가장 앞에 넣어주는 방식입니다. 

그 이후에 나온 T5의 구현체마다 미미하게 다른 방식을 씁니다. 만약 Downstream task가 '번역'만 다룬다면 굳이 필요 없을 수도 있습니다. 

본 실습에서는 Prefix도 넣어주는 방법으로 구현해봅니다.

In [16]:
max_token_length = 64

ENABLE_PREFIX = True

source_lang = "English"
target_lang = "Korean"

if ENABLE_PREFIX:
    prefix = f"translate {source_lang} to {target_lang}: "
else:
    prefix = ""
    
source_lang = source_lang.lower()
target_lang = target_lang.lower()
    
def preprocess_function(examples):
    inputs = [prefix + ex for ex in examples[source_lang]]
    targets = [ex for ex in examples[target_lang]]
    
    tokenized_examples = tokenizer(
                            inputs,
                            text_target=targets, 
                            max_length=max_token_length, 
                            truncation=True
                         )
    return tokenized_examples

### tokenizer test해보기

In [17]:
print(preprocess_function(dataset['train'][:2]).keys())
out = preprocess_function(dataset['train'][:2])
print(out)
print("-"*50)
print("Input Tokens  : ", out['input_ids'][0])
print("Input IDs     : ", tokenizer.decode(out['input_ids'][0]) )
print("-"*50)
print("Output Tokens : ", out['labels'][0])
print("Output IDs    : ", tokenizer.decode(out['labels'][0]) )

KeysView({'input_ids': [[35017, 6373, 10, 10698, 103, 388, 6809, 2952, 17, 8, 32204, 43, 8023, 6687, 28, 9495, 91, 3, 1], [35017, 6373, 10, 10698, 103, 4014, 322, 3170, 147, 67, 23274, 3, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[13946, 1800, 22, 4937, 4810, 19, 1657, 11023, 487, 35, 53, 26911, 131, 3, 1], [6842, 404, 951, 5767, 15387, 27, 831, 800, 4378, 15, 1587, 3, 1]]})
{'input_ids': [[35017, 6373, 10, 10698, 103, 388, 6809, 2952, 17, 8, 32204, 43, 8023, 6687, 28, 9495, 91, 3, 1], [35017, 6373, 10, 10698, 103, 4014, 322, 3170, 147, 67, 23274, 3, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[13946, 1800, 22, 4937, 4810, 19, 1657, 11023, 487, 35, 53, 26911, 131, 3, 1], [6842, 404, 951, 5767, 15387, 27, 831, 800, 4378, 15, 1587, 3, 1]]}
--------------------------------------------------
Input Tokens  :

### Dataset 전체에 걸쳐 전처리하기

In [18]:
tokenized_datasets = datasets.map(
                        preprocess_function, 
                        batched=True,
                        remove_columns=dataset["train"].column_names,
                     )

Map:   0%|          | 0/1040000 [00:00<?, ? examples/s]

Map:   0%|          | 0/130000 [00:00<?, ? examples/s]

Map:   0%|          | 0/130000 [00:00<?, ? examples/s]

In [19]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1040000
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 130000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 130000
    })
})

In [20]:
if USE_ALL_DATA: 
    #train_dataset = tokenized_datasets["train"]
    #valid_dataset = tokenized_datasets["validation"]
    #eval_dataset = tokenized_datasets["test"]
    train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(520000))
    valid_dataset = tokenized_datasets["validation"].shuffle(seed=42).select(range(6000)) # smaller
    eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(6000)) # smaller
else:    
    train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
    valid_dataset = tokenized_datasets["validation"].shuffle(seed=42).select(range(500))
    eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(500))

> 전체 데이터를 다 쓰게 되면, Colab 15G기준 10시간안에 실험이 안끝날 수 있습니다.
> half data를 써서 진행합니다.

## Model 준비 하기

In [21]:
from transformers import AutoModelForSeq2SeqLM
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_ID)

### Model Inference 수행해보기

In [22]:
from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

번역에 맞는 몇 가지 입출력 전처리를 추가합니다.
1. Batch 내 입력, 출력의 최대 길이 조정
2. Input뿐만 아니라 Output레이블에도 Padding을 추가해줍니다.
3. Decoder input 에 start symbol에 해당하는 pad를 넣어줍니다.

In [23]:
sample_batch = [tokenized_datasets["train"][i] for i in range(1, 3)]

In [24]:
print( sample_batch[0] )
print( sample_batch[1] )

{'input_ids': [35017, 6373, 10, 10698, 103, 83, 2155, 297, 92, 4632, 585, 46, 612, 3, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [70, 716, 2754, 38552, 1886, 55446, 3, 1]}
{'input_ids': [35017, 6373, 10, 10698, 103, 465, 43, 474, 267, 43, 16, 4271, 6934, 20, 1713, 16, 7920, 43, 35921, 117, 112, 4, 19757, 46, 437, 581, 4420, 5, 3352, 32296, 4, 13521, 8, 4, 13, 37945, 5, 1823, 14, 47253, 24498, 2119, 5, 17815, 14, 111, 3555, 3, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [5671, 22, 13582, 13728, 5356, 11681, 2359, 43159, 3905, 1559, 216, 1145, 903, 13744, 19693, 5197, 208, 6028, 2518, 9852, 5376, 3556, 39710, 501, 3, 1]}


In [25]:
batch = data_collator(sample_batch)

In [26]:
batch.keys()

KeysView({'input_ids': tensor([[35017,  6373,    10, 10698,   103,    83,  2155,   297,    92,  4632,
           585,    46,   612,     3,     1,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0],
        [35017,  6373,    10, 10698,   103,   465,    43,   474,   267,    43,
            16,  4271,  6934,    20,  1713,    16,  7920,    43, 35921,   117,
           112,     4, 19757,    46,   437,   581,  4420,     5,  3352, 32296,
             4, 13521,     8,     4,    13, 37945,     5,  1823,    14, 47253,
         24498,  2119,     5, 17815,    14,   111,  3555,     3,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0],
    

In [27]:
print( batch['input_ids'] )

tensor([[35017,  6373,    10, 10698,   103,    83,  2155,   297,    92,  4632,
           585,    46,   612,     3,     1,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0],
        [35017,  6373,    10, 10698,   103,   465,    43,   474,   267,    43,
            16,  4271,  6934,    20,  1713,    16,  7920,    43, 35921,   117,
           112,     4, 19757,    46,   437,   581,  4420,     5,  3352, 32296,
             4, 13521,     8,     4,    13, 37945,     5,  1823,    14, 47253,
         24498,  2119,     5, 17815,    14,   111,  3555,     3,     1]])


In [28]:
print( batch['attention_mask'] )

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1]])


In [29]:
print( batch['labels'] )

tensor([[   70,   716,  2754, 38552,  1886, 55446,     3,     1,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
          -100,  -100,  -100,  -100,  -100,  -100],
        [ 5671,    22, 13582, 13728,  5356, 11681,  2359, 43159,  3905,  1559,
           216,  1145,   903, 13744, 19693,  5197,   208,  6028,  2518,  9852,
          5376,  3556, 39710,   501,     3,     1]])


In [30]:
print( batch['decoder_input_ids'] )

tensor([[    0,    70,   716,  2754, 38552,  1886, 55446,     3,     1,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0],
        [    0,  5671,    22, 13582, 13728,  5356, 11681,  2359, 43159,  3905,
          1559,   216,  1145,   903, 13744, 19693,  5197,   208,  6028,  2518,
          9852,  5376,  3556, 39710,   501,     3]])


collator로 인해 2가지가 새로 변한 것을 알 수 있습니다.
- decoder_input_ids는 앞에 0 즉, pad symbol 이 추가되었습니다.
- labels 와 decoder_input_ids 를 살펴보면 하나씩 오른쪽으로 shift되었음을 확인할 수 있습니다.

In [31]:
#batch.to('cuda:0')
outputs = model(**batch)

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


In [32]:
outputs.keys()

odict_keys(['loss', 'logits', 'past_key_values', 'encoder_last_hidden_state'])

In [33]:
print(outputs.loss)
print(outputs.logits)

tensor(116.9538, grad_fn=<NllLossBackward0>)
tensor([[[ 1.8842, 16.9643, 17.4355,  ..., -2.5062,  1.9784,  1.8044],
         [-2.9827,  4.9538,  7.7252,  ..., -0.7923,  0.7647,  0.6146],
         [-1.3526,  6.5807,  8.1674,  ..., -1.0981,  0.8560,  0.8205],
         ...,
         [ 1.7649, 15.8479, 16.2174,  ..., -2.3402,  1.8477,  1.6772],
         [ 1.7653, 15.8521, 16.2221,  ..., -2.3408,  1.8482,  1.6777],
         [ 1.7653, 15.8540, 16.2245,  ..., -2.3411,  1.8484,  1.6779]],

        [[ 1.8084, 18.0851, 18.6995,  ..., -2.6947,  2.1221,  1.9421],
         [ 0.9778,  7.3106, 10.1147,  ..., -1.2494,  1.1158,  0.9796],
         [-0.7070,  6.3978,  8.7273,  ..., -1.2698,  1.0158,  0.8743],
         ...,
         [ 0.9110, 17.9524, 21.5942,  ..., -3.0746,  2.6767,  2.1209],
         [ 2.8914, 19.4854, 23.0626,  ..., -3.2397,  2.6137,  2.0192],
         [ 1.6808, 11.7603, 12.2126,  ..., -2.0348,  1.6106,  1.0296]]],
       grad_fn=<UnsafeViewBackward0>)


### 예측 결과를 Symbol 로 바꾸기

In [34]:
import torch
best_output = torch.argmax(outputs['logits'][0], axis=1).cpu().numpy() 
tokenizer.convert_ids_to_tokens( best_output )

['큐브',
 '큐브',
 '▁비일비재',
 '큐브',
 '큐브',
 '▁비일비재',
 '큐브',
 '▁비일비재',
 '▁비일비재',
 '큐브',
 '큐브',
 '큐브',
 '큐브',
 '큐브',
 '큐브',
 '큐브',
 '큐브',
 '큐브',
 '큐브',
 '큐브',
 '큐브',
 '큐브',
 '큐브',
 '큐브',
 '큐브',
 '큐브']

어떻게든 답은 나오지만, 제대로 된 결과가 아님을 알 수 있습니다. 훈련을 통해 제대로 된 결과를 얻어보죠.

## Fine-Tuning 수행하기

### Evaluation Metric 

In [35]:
import evaluate
metric = evaluate.load("sacrebleu")

### SacreBLEU
SacreBLEU는 자연어 처리(NLP) 분야에서 기계 번역의 품질을 평가하기 위해 널리 사용되는 도구입니다. BLEU(BiLingual Evaluation Understudy) 점수는 기계 번역의 출력과 사람이 번역한 참조 번역 사이의 유사도를 측정하는 방법 중 하나로, SacreBLEU는 이 BLEU 점수를 보다 일관되고 신뢰할 수 있는 방식으로 계산합니다.

In [36]:
import numpy as np

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    
    if isinstance(preds, tuple):
        preds = preds[0]
    
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}
    
    return result


위의 compute_metrics 는 [Transformer 번역 예제](https://huggingface.co/course/chapter7/4?fw=pt#metrics)에서 가져왔습니다.

### Huggingface 의 Trainer 활용하여 훈련하기

In [37]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    f"{MODEL_ID}-lora-finetuned-english-to-korean",
    ## ---- Epoch & Batch ----- ##
    num_train_epochs=3,             # 훈련 에폭 수
    #num_train_epochs=10,            # 훈련 에폭 수
    #max_steps=100,

    per_device_train_batch_size=30,  # 디바이스 당 훈련 배치 크기 (to be 20)
    auto_find_batch_size=True, 
    
    ## ---- Learning Rate ----- ##
    #warmup_steps=500,               # 워밍업을 위한 스텝 수
    weight_decay=0.01,               # 가중치 감소율
    learning_rate=1e-3,              # 데이터 셋 마다 적정값을 찾아야 합니다.
    
    ## - Logging & Checkpoint - ##
    save_strategy="epoch",           # 에폭마다 모델을 저장
    logging_steps=10,                # 로그를 기록할 스텝 간격
    
    ## ---- Evaluations ----- ##
    predict_with_generate=True,
    eval_strategy="epoch",
)


- fp16=False is default value

### Trainer 설정하기

In [38]:
from transformers import Seq2SeqTrainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    
    # on Which Dataset
    train_dataset=train_dataset, # < train
    eval_dataset=valid_dataset,   # < test
    
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/tmp/ipykernel_7416/4111572105.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [39]:
import warnings

# huggingface 훈련 중 불필요한 Warning 이 많이 발생함 (epoch 별로)
warnings.filterwarnings("ignore", category=UserWarning)

In [40]:
#torch.cuda.empty_cache()

In [ ]:
trainer.train()

In [ ]:
trainer.save_model("./results")

In [ ]:
# finish a 'run' session
wandb.finish()

## 훈련된 모델 Inference

In [ ]:
model_dir = "./results"
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)

model.cpu();

In [ ]:
input_text = ['translate English to Korean : '+x for x in datasets['validation']['english'][:2] ]

In [ ]:
#input_text = datasets['validation']['english'][:2] 

In [ ]:
inputs = tokenizer(input_text, 
                   return_tensors="pt", 
                   padding=True, 
                   max_length=max_token_length)

In [ ]:
tokenizer.decode(inputs['input_ids'][0])

In [ ]:
tokenizer.decode(inputs['input_ids'][1])

In [ ]:
machines = model.generate(
    **inputs,
    max_length=max_token_length,
    num_beams=5,
)

machines.shape

In [ ]:
[ 
    tokenizer.convert_tokens_to_string(
    tokenizer.convert_ids_to_tokens(m)) for m in machines
]